## Intro to Supervised Machine Learning: Housing Prices Complete Case Study

You work for a consultancy that specializes in real estate: your clients involve developers, agencies and investors.  It's a domain of home appraisers, who aim to determine the value of a house in an unbiased way, and follow an official criteria to ensure they don't play in favour of neither the buyer or the seller.

A central aspect to the business is pricing. However, the market often deviates from appraisal prices. Your consultancy wants to detect these deviations. It has therefore acquired a dataset containing a historical register of housing prices - the actual prices at which the properties were sold. For each house, in addition to price, the data contains around 80 different features, such as the area, the state of the property, whether it has a backyard or not, and so on.

You were tasked with creating a model that predicts the prices. Your manager told you that the model should have an average relative error of 15%. With such a model, you would be able to advise your clients on housing in a massively scalable way: once the features of a house have been collected, the model will output the price. The results of your work are below...


# Reading & Exploring

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/OLD_CURR/UNIT7/DAY1/housing_prices.csv")

In [ ]:
# Get a glimpse of the dataframe
data.head()

In [ ]:
# Get the shape of the dataframe
data.shape

In [ ]:
# Get the colum types.
data.info()

In [ ]:
# Get a histogram of selling prices which is what we want to predict
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import matplotlib.ticker as ticker
import seaborn as sns
%matplotlib inline

g = sns.displot(data['SalePrice'],height=8.27, aspect=1)
for ax in g.axes.flat:
    ax.xaxis.set_major_locator(ticker.MultipleLocator(50000))
    ax.xaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{int(x/1000)}K'))


Clearly, tihs distribution is highly skewed, threrefore it's needs to be transformed. Dependent variables which are highly skewed can be easilly transformed taking the logarithm.

In [ ]:
g = sns.displot(np.log(data['SalePrice']),height=8.27, aspect=1)

In [ ]:
def plot_count_box(df, grouping_var, dependent_var, orient = 'v'):
    '''
    Function to plot two plots side by side.
    The left plot will be a count plot of 'grouping_var'
    The right plot will be a boxplot of 'dependent_var' grouped by 'grouping var'
    Inputs: 
    df = dataframe
    grouping_var = variable to perform the agregation
    dependent_var = variable to plot the boxplot
    Output:
    A canvas with a counplot on the left and boxplot on the right
    The boxplots are ordered based on the median value of the dependent feature.
    '''

    # Getting a list of unique valus for the grouping variable
    levels = list(df[grouping_var].unique())    
    diver = sns.color_palette("Paired", len(levels)) # RdBu, Set3, Paired
    # Setting a dictionary of {'value': color} to use in each plot
    cmap = dict(zip(levels,diver))

    mean_price = pd.DataFrame(df.groupby([grouping_var])[dependent_var].median().sort_values()).reset_index()

    fig, ax = plt.subplots(1,2,figsize=(16,8))
    if ( orient == 'v' ):
        sns.countplot(x = grouping_var, data = df, order = df[grouping_var].value_counts().index, ax = ax[0], palette = cmap) 
        sns.boxplot(y= dependent_var , x= grouping_var, data = data, order = mean_price[grouping_var], ax = ax[1], palette=  cmap)
        ax[1].yaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{int(x/1000)}K'))
    else:
        sns.countplot(y = grouping_var, data = df, order = df[grouping_var].value_counts().index, ax = ax[0], palette = cmap)
        sns.boxplot(x= dependent_var , y= grouping_var, data = data, order = mean_price[grouping_var], ax = ax[1], palette = cmap)
        ax[1].xaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{int(x/1000)}K'))        
    fig.show()
    pass

MSZoning: Identifies the general zoning classification of the sale.
		
       A	Agriculture
       C	Commercial
       FV	Floating Village Residential
       I	Industrial
       RH	Residential High Density
       RL	Residential Low Density
       RP	Residential Low Density Park 
       RM	Residential Medium Density

In [ ]:
# Check out the variable "MSZoning"

plot_count_box(data, 'MSZoning', 'SalePrice')

MSZoning is a highly imbalanced variable: we have much more samples of type RL than the others. That prevents to make a fair comparison of the boxplots of selling prices split by this variable. In addition, even with the small size of the dataset there is clear overlap between the median SalesPrices for each value of MSZooning. Even more, the houses in the most sample MSZooning has has huge rangfe of values which doesn't allow to discriminate between the other values. Therefore, this variable doesn't look quite usefull to predit the SalesPrice.

In [ ]:
cols_to_drop = []
cols_to_drop += ['MSZoning']

Neighborhood: Physical locations within Ames city limits

       Blmngtn	Bloomington Heights
       Blueste	Bluestem
       BrDale	Briardale
       BrkSide	Brookside
       ClearCr	Clear Creek
       CollgCr	College Creek
       Crawfor	Crawford
       Edwards	Edwards
       Gilbert	Gilbert
       IDOTRR	Iowa DOT and Rail Road
       MeadowV	Meadow Village
       Mitchel	Mitchell
       Names	North Ames
       NoRidge	Northridge
       NPkVill	Northpark Villa
       NridgHt	Northridge Heights
       NWAmes	Northwest Ames
       OldTown	Old Town
       SWISU	South & West of Iowa State University
       Sawyer	Sawyer
       SawyerW	Sawyer West
       Somerst	Somerset
       StoneBr	Stone Brook
       Timber	Timberland
       Veenker	Veenker

In [ ]:
# Check out the variable "Neighbourhood"

plot_count_box(data, 'Neighborhood', 'SalePrice', orient = 'h')

In [ ]:
len(list(data['Neighborhood'].unique()))

We have a similar problem as before. To make it worse, the `cardinality` (number of possible values) of this variable is huge (25). Therefore, we can think of dropping this variable.

In [ ]:
cols_to_drop += ['Neighborhood']

OverallQual: Rates the overall material and finish of the house

       10	Very Excellent
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average
       5	Average
       4	Below Average
       3	Fair
       2	Poor
       1	Very Poor

In [ ]:
# Check out the variable "OverallQual"

plot_count_box(data, 'OverallQual', 'SalePrice')

Again, we have a class imbalance. However, as it can be seen in the above picture the overal quality has a big impact on the SalePrice as it could be expected. This variable looks promising and the relationships between the OveralQual and the SalesPrice seems to be quadratic. Let's test this assumption

In [ ]:
sns.regplot(x = data['OverallQual']**2, y = data['SalePrice'])

We see vertical bars because this variable is a perfect example of an ordinal categorical variable.

OverallCond: Rates the overall condition of the house

       10   Elite
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average	
       5	Average
       4	Below Average	
       3	Fair
       2	Poor
       1	Very Poor

In [ ]:
# Check out the variable "OverallCond"

plot_count_box(data, "OverallCond", "SalePrice")

Again we also have a class imbalance and there is a high range of selling prices for every possible value of `OverallCond`. In addition, this variable can be redundant as we already have the previous one. Therefore, we will drop this one.

In [ ]:
cols_to_drop += ['OverallCond']

MoSold: Month Sold (MM)


In [ ]:
# Check out the variable "MoSold"

plot_count_box(data, "MoSold", "SalePrice")

Selling month doesn't have any relationship with the selling price, even is worth of knowing that the month with the highest amount of sales is june. Therefore, let's drop the variable `MoSold`

In [ ]:
cols_to_drop += ['MoSold']

In [ ]:
# Check out the variable "LotArea"which is continuous

levels = list(data['OverallQual'].unique())  # OverallCond  
diver = sns.color_palette("RdBu", len(levels)) # RdBu, Set3, Paired
# Setting a dictionary of {'value': color} to use in each plot
cmap = dict(zip(levels,diver))

fig, ax = plt.subplots(1,2,figsize=(16,8))
sns.distplot(data['LotArea'], ax = ax[0], kde=False)
sns.scatterplot(x = 'LotArea', y = 'SalePrice', data = data, ax = ax[1], hue ='OverallQual', palette=cmap) # hue ='OverallCond'
ax[0].xaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{int(x/1000)}K'))
ax[1].yaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{int(x/1000)}K'))
ax[1].xaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{int(x/1000)}'))
ax[1].set_xlabel('LotArea (Squared feet x 1000)')
ax[1].axvline(x=25000)



Clearly, the distribution of squared feet is strongly skewed. On the other hamd it seems that below 25K the relationship between the selling price and the squared feet seems to be linnear. In contrast, for higher surface values, the is no clear relationship between the selling price and the surface.

Another conclussion that can be extracted from the previous plots is that above 25K squared feets, the overall quality is usually above 5.

Let's check the previous assumption

In [ ]:
sns.scatterplot(x ='LotArea', y = 'SalePrice', data = data[data['LotArea'] < 25000], hue ='OverallQual', palette=cmap)

In [ ]:
sns.displot(data[data['LotArea'] < 25000 ]['LotArea'], kde=False)

We can apply this filter to our dataframe

In [ ]:
data = data[data['LotArea'] < 25000 ]

# Finding missing values

In [ ]:
# Get a dataframe of missing values by column. Use sort_values(ascending=False) to sort the results
nulls = pd.DataFrame(data.isnull().sum().sort_values(ascending=False))


# Use reset_index() to make the index to become column and rename the column names
nulls.reset_index(inplace=True)
nulls.columns = ['Variable','Count']
nulls.head()

In [ ]:
# Get the columns which have NA's

nulls[nulls['Count'] > 0].Variable

Let's start dealing with those NA's column by column

PoolQC: Pool quality
		
       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       NA	No Pool

If the is no pool, we can assing a value flag to it like 'NA'

In [ ]:
data['PoolQC'] = data['PoolQC'].fillna('NA')
data['PoolQC'].unique()

Now let's inspect the relatioship between `PoolQC`and `SellPrice`



In [ ]:
data['PoolQC'].value_counts()

In [ ]:
plot_count_box(data[data['PoolQC'] != 'NA'], "PoolQC", "SalePrice")

Clearly, the sampling of houses with pool is not big enough to extract conclussions. Therefore we can drop this feature

In [ ]:
cols_to_drop += ['PoolQC']

MiscFeature: Miscellaneous feature not covered in other categories
		
       Elev	Elevator
       Gar2	2nd Garage (if not described in garage section)
       Othr	Other
       Shed	Shed (over 100 SF)
       TenC	Tennis Court
       NA	None

In [ ]:
data['MiscFeature'].value_counts()

In [ ]:
plot_count_box(data, "MiscFeature", "SalePrice")

Clearly we don't have enough samples to extract conclussions. Therefore, we will drop this other feature.

In [ ]:
cols_to_drop += ['MiscFeature']

Alley: Type of alley access to property

       Grvl	Gravel
       Pave	Paved
       NA 	No alley access

In [ ]:
plot_count_box(data, "Alley", "SalePrice")

It seems to be some overlap between both possibilities although having a pavement alley usually has a bigger impact and the sampling is poor. Again we will drop this feature.

In [ ]:
cols_to_drop += ['Alley']

Fence: Fence quality
		
       GdPrv	Good Privacy
       MnPrv	Minimum Privacy
       GdWo	Good Wood
       MnWw	Minimum Wood/Wire
       NA	No Fence

In [ ]:
plot_count_box(data, "Fence", "SalePrice")

Clearly it looks like that there are no significant differences among the fence condition. Therefore this variable is a good candidate to drop.

In [ ]:
cols_to_drop += ['Fence']

FireplaceQu: Fireplace quality

       Ex	Excellent - Exceptional Masonry Fireplace
       Gd	Good - Masonry Fireplace in main level
       TA	Average - Prefabricated Fireplace in main living area or Masonry Fireplace in basement
       Fa	Fair - Prefabricated Fireplace in basement
       Po	Poor - Ben Franklin Stove
       NA	No Fireplace

In [ ]:
data['FireplaceQu'] = data['FireplaceQu'].fillna('NA')

In [ ]:
plot_count_box(data, "FireplaceQu", "SalePrice")

In [ ]:
data['FireplaceQu'].value_counts()

Clearly, the fireplace condition ( when it's present ) can have an impact. Let's test this. 

In [ ]:
print("Mean price of houses with fireplace: ",round(data[data['FireplaceQu'] != 'NA']['SalePrice'].mean()))
print("Mean price od houses without fireplace: ",round(data[data['FireplaceQu'] == 'NA']['SalePrice'].mean()))
y = data[data['FireplaceQu'] != 'NA']['SalePrice'].mean()
n = data[data['FireplaceQu'] == 'NA']['SalePrice'].mean()
print("Average price increase by having a fireplace: {} %".format(round(((y/n)-1)*100)))

Therefore, let's make this feature binnary.

In [ ]:
data.rename(columns={'FireplaceQu': 'Fireplace'}, inplace = True)
data['Fireplace'] = np.where(data['Fireplace'].isin(['Gd','TA','Fa','Ex','Po']),1,0)

LotFrontage: Linear feet of street connected to property

In [ ]:
sns.scatterplot(x = 'LotFrontage',y = 'SalePrice', data = data)
data[['LotFrontage','SalePrice']].corr()

There is no clear linear relationship between both features. Therefore, we will drop the feature.

In [ ]:
cols_to_drop += ["LotFrontage"]

6. GarageCond
7. GarageType
8. GarageYrBlt
9. GarageFinish
10. GarageQual

All those variables seem redundant on a first look. Let's inspect them and pick the best.

In [ ]:
my_list = ['GarageCond','GarageType','GarageFinish','GarageQual']

for elem in my_list:
    data[elem] = data[elem].fillna('NA')
    plot_count_box(data, elem, "SalePrice")

`GarageCond` is heavily unbalanced and thre is high overlap between the boxplots for all the values. We can drop this feature.

`GarageType` is heavyly unbalanced which makes the statistics poor and there is too much overlap between the types to extract conclussions. We can drop this feature.

`GarageFinish` has enough samples of each type but there is significant overlap between the boxplots, even though it seems to increase slightly the selling price.

`GarageQual` is also heavily unbalanced and seems to be quite similar to be redundant. We can think of dropping it.

It looks like having a garage impact but those characteristics seem unclear.

In [ ]:
cols_to_drop += ['GarageCond','GarageType','GarageQual']

Let's inspect `GarageYrBlt`


In [ ]:
sns.scatterplot(x = 'GarageYrBlt', y = 'SalePrice', data = data)

It looks like that the Sale Price increases over time, although not linearly. It would be more interesting to know in which year the dataset was collected in order to know how much lasted since the last renewal. It looks more reasonable to compute the difference in terms of mean price between those house which have a garage and don't. Hoewever, a house can have a garage but a missing value in this field. This will bias conclussions.

In [ ]:
y = data[data['GarageYrBlt'].isna() == True]['SalePrice'].mean()
n = data[data['GarageYrBlt'].isna() == False]['SalePrice'].mean()
print("Mean price of houses with garage: ",round(y))
print("Mean price od houses without garave: ",round(n))

print("Average price increase by having a garage: {} %".format(round(((y/n)-1)*100)))

In [ ]:
cols_to_drop += ['GarageYrBlt']

11. BsmtExposure
12. BsmtFinType2
13. BsmtFinType1
14. BsmtCond
15. BsmtQual

BsmtQual: Evaluates the height of the basement

* Ex	Excellent (100+ inches)	
* Gd	Good (90-99 inches)
* TA	Typical (80-89 inches)
* Fa	Fair (70-79 inches)
* Po	Poor (<70 inches)
* NA	No Basemen

BsmtCond: Evaluates the general condition of the basement

* Ex	Excellent
* Gd	Good
* TA	Typical - slight dampness allowed
* Fa	Fair - dampness or some cracking or settling
* Po	Poor - Severe cracking, settling, or wetness
* NA	No Basement

BsmtExposure: Refers to walkout or garden level walls

* Gd	Good Exposure
* Av	Average Exposure (split levels or foyers typically score average or above)	
* Mn	Mimimum Exposure
* No	No Exposure
* NA	No Basement

BsmtFinType1: Rating of basement finished area

* GLQ	Good Living Quarters
* ALQ	Average Living Quarters
* BLQ	Below Average Living Quarters	
* Rec	Average Rec Room
* LwQ	Low Quality
* Unf	Unfinshed
* NA	No Basement

BsmtFinSF1: Type 1 finished square feet

BsmtFinType2: Rating of basement finished area (if multiple types)

* GLQ	Good Living Quarters
* ALQ	Average Living Quarters
* BLQ	Below Average Living Quarters	
* Rec	Average Rec Room
* LwQ	Low Quality
* Unf	Unfinshed
* NA	No Basement

BsmtFinSF2: Type 2 finished square feet

BsmtUnfSF: Unfinished square feet of basement area

TotalBsmtSF: Total square feet of basement area

In [ ]:
my_list = ['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2']

for elem in my_list:
    data[elem] = data[elem].fillna('NA')
    plot_count_box(data, elem, "SalePrice")

`BsmtQual` is heavily imbalanced. The biggest groups have a big overlap, therefore those groups can be groupped together in a single group.

`BsmCons`is also heavily unbalanced to extract conclussions.

`BsmtExposure` values have a big overlap.

`BsmtFinType1` and `BsmtFinType2` have too much overlap between the possible values. In particular, for the last value there is a high class imbalance.

All those features have high overlap, therefore they are not informative. Let's drop them.

In [ ]:
cols_to_drop += ['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2']

BsmtFinSF2: Type 2 finished square feet

BsmtUnfSF: Unfinished square feet of basement area

TotalBsmtSF: Total square feet of basement area

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(20,8))
sns.scatterplot(x= 'BsmtFinSF2' , y= 'SalePrice', data = data,  ax = ax[0])
sns.scatterplot(x= 'BsmtUnfSF' , y= 'SalePrice', data = data,  ax = ax[1])
sns.scatterplot(x= 'TotalBsmtSF', y= 'SalePrice', data = data,  ax = ax[2])
ax[0].yaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{int(x/1000)}K'))
ax[1].yaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{int(x/1000)}K'))
ax[2].yaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{int(x/1000)}K'))

According to the previous figure, it only seems to be a relationship between `TotalBsmtSF` and `SalePrice`.

In [ ]:
cols_to_drop += ['BsmtFinSF2','BsmtUnfSF']

In [ ]:
data['TotalBsmtSF'] = data['TotalBsmtSF'].fillna(0) 
data[['TotalBsmtSF','SalePrice']].corr()

MasVnrType: Masonry veneer type

       BrkCmn	Brick Common
       BrkFace	Brick Face
       CBlock	Cinder Block
       None	None
       Stone	Stone
	
MasVnrArea: Masonry veneer area in square feet

In [ ]:
data['MasVnrType'].value_counts(dropna=False)
data['MasVnrType'] = data['MasVnrType'].fillna('None')

In [ ]:
# MasVnrType

plot_count_box(data, 'MasVnrType', "SalePrice")

Too much overlap and imbalance and overlap. Let's drop this feature.

In [ ]:
cols_to_drop += ['MasVnrType']

In [ ]:
# MasVnrArea

sns.scatterplot( x = 'MasVnrArea', y = 'SalePrice', data = data)

Not relation at all. Another feature to drop.

In [ ]:
cols_to_drop += ['MasVnrArea']

Electrical: Electrical system

       SBrkr	Standard Circuit Breakers & Romex
       FuseA	Fuse Box over 60 AMP and all Romex wiring (Average)	
       FuseF	60 AMP Fuse Box and mostly Romex wiring (Fair)
       FuseP	60 AMP Fuse Box and mostly knob & tube wiring (poor)
       Mix	Mixed

In [ ]:
data['Electrical'] = data['Electrical'].fillna('NA') 

plot_count_box(data, 'Electrical', "SalePrice")

Another example of a feature with high class imbalance and high overlap between the boxplots.

In [ ]:
cols_to_drop += ['Electrical']

In [ ]:
cols_to_drop

# Dropping unwanted columns

In [ ]:
data.drop(columns = cols_to_drop, axis = 1, inplace = True)
data.info()

In [ ]:
data.shape

# Inspecting categorical features

'Street'

In [ ]:
data['Street'].value_counts()

High class imbalance, let's drop it.

In [ ]:
cols_to_drop = ['Street']

LotShape: General shape of property

       Reg	Regular	
       IR1	Slightly irregular
       IR2	Moderately Irregular
       IR3	Irregular

In [ ]:
plot_count_box(data, 'LotShape', 'SalePrice')

It doesn't seem to have a impact on the sale price. Let's drop it.

In [ ]:
cols_to_drop += ['LotShape']

LandContour: Flatness of the property

       Lvl	Near Flat/Level	
       Bnk	Banked - Quick and significant rise from street grade to building
       HLS	Hillside - Significant slope from side to side
       Low	Depression

In [ ]:
plot_count_box(data, 'LandContour', 'SalePrice')

No clear differences and high class imbalance within the possible values. Another feature to drop.

In [ ]:
cols_to_drop += ['LandContour']

Utilities: Type of utilities available
		
       AllPub	All public Utilities (E,G,W,& S)	
       NoSewr	Electricity, Gas, and Water (Septic Tank)
       NoSeWa	Electricity and Gas Only
       ELO	Electricity only

In [ ]:
plot_count_box(data, 'Utilities', 'SalePrice')

Huge class imbalance, therefore this feature is not informative.

In [ ]:
cols_to_drop += ['Utilities']

LotConfig: Lot configuration

       Inside	Inside lot
       Corner	Corner lot
       CulDSac	Cul-de-sac
       FR2	Frontage on 2 sides of property
       FR3	Frontage on 3 sides of property

In [ ]:
plot_count_box(data, 'LotConfig', 'SalePrice')

High class imbalance and not informative.

In [ ]:
cols_to_drop += ['LotConfig']

LandSlope: Slope of property
		
       Gtl	Gentle slope
       Mod	Moderate Slope	
       Sev	Severe Slope

In [ ]:
plot_count_box(data, 'LandSlope', 'SalePrice')

Same as well

In [ ]:
cols_to_drop += ['LandSlope']

Condition1: Proximity to various conditions
	
       Artery	Adjacent to arterial street
       Feedr	Adjacent to feeder street	
       Norm	Normal	
       RRNn	Within 200' of North-South Railroad
       RRAn	Adjacent to North-South Railroad
       PosN	Near positive off-site feature--park, greenbelt, etc.
       PosA	Adjacent to postive off-site feature
       RRNe	Within 200' of East-West Railroad
       RRAe	Adjacent to East-West Railroad

In [ ]:
plot_count_box(data, 'Condition1', 'SalePrice')

Small statistics to extract conclussions. Let's drop it.

In [ ]:
cols_to_drop += ['Condition1']

Condition2: Proximity to various conditions (if more than one is present)
		
       Artery	Adjacent to arterial street
       Feedr	Adjacent to feeder street	
       Norm	Normal	
       RRNn	Within 200' of North-South Railroad
       RRAn	Adjacent to North-South Railroad
       PosN	Near positive off-site feature--park, greenbelt, etc.
       PosA	Adjacent to postive off-site feature
       RRNe	Within 200' of East-West Railroad
       RRAe	Adjacent to East-West Railroad

In [ ]:
plot_count_box(data, 'Condition2', 'SalePrice')

Even worse.

In [ ]:
cols_to_drop += ['Condition2']

BldgType: Type of dwelling
		
       1Fam	Single-family Detached	
       2FmCon	Two-family Conversion; originally built as one-family dwelling
       Duplx	Duplex
       TwnhsE	Townhouse End Unit
       TwnhsI	Townhouse Inside Unit

In [ ]:
plot_count_box(data, 'BldgType', 'SalePrice')

Again, high imbalanace and high overlap. A pity as this could be a good feature.

In [ ]:
cols_to_drop += ['BldgType']

HouseStyle: Style of dwelling
	
       1Story	One story
       1.5Fin	One and one-half story: 2nd level finished
       1.5Unf	One and one-half story: 2nd level unfinished
       2Story	Two story
       2.5Fin	Two and one-half story: 2nd level finished
       2.5Unf	Two and one-half story: 2nd level unfinished
       SFoyer	Split Foyer
       SLvl	Split Level

In [ ]:
plot_count_box(data, 'HouseStyle', 'SalePrice')

Same problem

In [ ]:
cols_to_drop += ['HouseStyle']

RoofStyle: Type of roof

       Flat	Flat
       Gable	Gable
       Gambrel	Gabrel (Barn)
       Hip	Hip
       Mansard	Mansard
       Shed	Shed

In [ ]:
plot_count_box(data, 'RoofStyle', 'SalePrice')

Same

In [ ]:
cols_to_drop += ['RoofStyle']

RoofMatl: Roof material

       ClyTile	Clay or Tile
       CompShg	Standard (Composite) Shingle
       Membran	Membrane
       Metal	Metal
       Roll	Roll
       Tar&Grv	Gravel & Tar
       WdShake	Wood Shakes
       WdShngl	Wood Shingles

In [ ]:
plot_count_box(data, 'RoofMatl', 'SalePrice')

Apperently WdShngl houses have a higher sale price but the number of samples is nog big enough

In [ ]:
data['RoofMatl'].value_counts()

Useless feature because of lack of data

In [ ]:
cols_to_drop += ['RoofMatl']

Exterior1st: Exterior covering on house

       AsbShng	Asbestos Shingles
       AsphShn	Asphalt Shingles
       BrkComm	Brick Common
       BrkFace	Brick Face
       CBlock	Cinder Block
       CemntBd	Cement Board
       HdBoard	Hard Board
       ImStucc	Imitation Stucco
       MetalSd	Metal Siding
       Other	Other
       Plywood	Plywood
       PreCast	PreCast	
       Stone	Stone
       Stucco	Stucco
       VinylSd	Vinyl Siding
       Wd Sdng	Wood Siding
       WdShing	Wood Shingles

In [ ]:
plot_count_box(data, 'Exterior1st', 'SalePrice', orient = 'h')

We have some values with a lot of samples and others undersampled. We can can think groupping the low sampled groups together.



In [ ]:
data['Exterior1st'].value_counts()

In [ ]:
data['Exterior1st'] = np.where(data['Exterior1st'].isin(['VinylSd','MetalSd','HdBoard','Wd Sdng','Plywood']),data['Exterior1st'],'Other')

In [ ]:
data['Exterior1st'].value_counts()

In [ ]:
plot_count_box(data, 'Exterior1st', 'SalePrice', orient = 'h')

Even grouping, this variable doesn't seem to help in determining the final selling prices given the overlap.

In [ ]:
cols_to_drop += ['Exterior1st']

Exterior2nd: Exterior covering on house (if more than one material)

       AsbShng	Asbestos Shingles
       AsphShn	Asphalt Shingles
       BrkComm	Brick Common
       BrkFace	Brick Face
       CBlock	Cinder Block
       CemntBd	Cement Board
       HdBoard	Hard Board
       ImStucc	Imitation Stucco
       MetalSd	Metal Siding
       Other	Other
       Plywood	Plywood
       PreCast	PreCast
       Stone	Stone
       Stucco	Stucco
       VinylSd	Vinyl Siding
       Wd Sdng	Wood Siding
       WdShing	Wood Shingles

In [ ]:
plot_count_box(data, 'Exterior2nd', 'SalePrice', orient = 'h')

In [ ]:
data['Exterior2nd'].value_counts()

It seems to be redundant. Let's drop it.

In [ ]:
cols_to_drop += ['Exterior2nd']

ExterQual: Evaluates the quality of the material on the exterior 
		
       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       Po	Poor

In [ ]:
plot_count_box(data, 'ExterQual', 'SalePrice')

In [ ]:
data['ExterQual'].value_counts()

In [ ]:
m1 = data[data['ExterQual'] == 'Fa']['SalePrice'].mean()
m2 = data[data['ExterQual'].isin(['TA','Gd']) == True]['SalePrice'].mean()
m3 = data[data['ExterQual'] == 'Ex']['SalePrice'].mean()

print("Mean prices of houses with ExterQual = 'Fa': {:.2f}".format(m1))
print("Mean prices of houses with ExterQual = ['TA','Gd']: {:.2f}".format(m2))
print("Mean prices of houses with ExterQual = 'Ex': {:.2f}".format(m3))

print("Ratio m2/m1: {:.2f}".format(m2/m1))
print("Ratio m2/m1: {:.2f}".format(m3/m1))

In [ ]:
#data['ExterQual'] = np.where(data['ExterQual'].isin(['TA','Gd']),'TA-Gd',data['ExterQual'])
data['ExterQual'] = data['ExterQual'].replace({'Fa':1, 'TA': 1.97,'Gd': 1.97, 'Ex': 4.23})

ExterCond: Evaluates the present condition of the material on the exterior
		
       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       Po	Poor

In [ ]:
plot_count_box(data, 'ExterCond', 'SalePrice')

The most sampled value for 'ExterCond' is 'TA' which has a huge range of selling prices. Let's drop it

In [ ]:
cols_to_drop += ['ExterCond']

Foundation: Type of foundation
		
       BrkTil	Brick & Tile
       CBlock	Cinder Block
       PConc	Poured Contrete	
       Slab	Slab
       Stone	Stone
       Wood	Wood

In [ ]:
plot_count_box(data, 'Foundation', 'SalePrice')

No clear conclussions can be extracted

In [ ]:
cols_to_drop += ['Foundation']

Heating: Type of heating
		
       Floor	Floor Furnace
       GasA	Gas forced warm air furnace
       GasW	Gas hot water or steam heat
       Grav	Gravity furnace	
       OthW	Hot water or steam heat other than gas
       Wall	Wall furnace

In [ ]:
plot_count_box(data, 'Heating', 'SalePrice')

Not conclussive 

In [ ]:
data['Heating'].value_counts()

In [ ]:
cols_to_drop += ['Heating']

HeatingQC: Heating quality and condition

       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       Po	Poor

In [ ]:
plot_count_box(data, 'HeatingQC', 'SalePrice')

Good sampling ( except 'poor' ) but high overlap to be conclussive. Let's drop it. 

In [ ]:
cols_to_drop += ['HeatingQC']

CentralAir: Central air conditioning

       N	No
       Y	Yes

In [ ]:
plot_count_box(data, 'CentralAir', 'SalePrice')

In [ ]:
data['CentralAir'].value_counts()

I will keep this feature as even though there is some overlap, in general the selling prices are bigger for houses where CentralAir is present. However, I will recode this feature.

In [ ]:
data['CentralAir'] = np.where(data['CentralAir'] == "Y",1,0)

KitchenQual: Kitchen quality

       Ex	Excellent
       Gd	Good
       TA	Typical/Average
       Fa	Fair
       Po	Poor

In [ ]:
plot_count_box(data,'KitchenQual', 'SalePrice')

In [ ]:
data['KitchenQual'].value_counts()

In [ ]:
m1 = data[data['KitchenQual'] == 'Fa']['SalePrice'].mean()
m2 = data[data['KitchenQual'].isin(['TA','Gd']) == True]['SalePrice'].mean()
m3 = data[data['KitchenQual'] == 'Ex']['SalePrice'].mean()

print("Mean price of houses where KitchenQual = 'Fa', {}".format(round(m1,2)))
print("Mean price of houses where KitchenQual = ['TA','Gd'], {}".format(round(m2,2)))
print("Mean price of houses where KitchenQual = 'Ex', {}".format(round(m3,2)))

print("Ratio m2/m1: ",round(m2/m1,2))
print("Ratio m3/m1: ",round(m3/m1,2))

Sampling is good but the groups can be grouped in two groups:

* Group 1: Ex/Gd
* Group 2: Else

In [ ]:
#data['KitchenQual'] = np.where(data['KitchenQual'].isin(['Fa','Po','TA']),0,1)
data['KitchenQual'] = data['KitchenQual'].replace({'Fa': 1, 'TA': 1.61, 'Gd': 1.61, 'Ex': 3.14})

Functional: Home functionality (Assume typical unless deductions are warranted)

       Typ	Typical Functionality
       Min1	Minor Deductions 1
       Min2	Minor Deductions 2
       Mod	Moderate Deductions
       Maj1	Major Deductions 1
       Maj2	Major Deductions 2
       Sev	Severely Damaged
       Sal	Salvage only

In [ ]:
plot_count_box(data,'Functional', 'SalePrice')

Terrible sampling and high overlap. Feature to drop

In [ ]:
cols_to_drop += ['Functional']

GarageFinish: Interior finish of the garage

       Fin	Finished
       RFn	Rough Finished	
       Unf	Unfinished
       NA	No Garage

In [ ]:
plot_count_box(data,'GarageFinish', 'SalePrice')

Good sampling but mnot conclussive bercause of overlap. Probably is best to have a feature between having a garage or not. Moreover, there are other numerical features which takes into account the number of cars and the surface. We will drop this feature.

In [ ]:
cols_to_drop += ['GarageFinish']

PavedDrive: Paved driveway

       Y	Paved 
       P	Partial Pavement
       N	Dirt/Gravel

In [ ]:
plot_count_box(data,'PavedDrive', 'SalePrice')

High overlap. Feature to drop.

In [ ]:
cols_to_drop += ['PavedDrive']

SaleType: Type of sale
		
       WD 	Warranty Deed - Conventional
       CWD	Warranty Deed - Cash
       VWD	Warranty Deed - VA Loan
       New	Home just constructed and sold
       COD	Court Officer Deed/Estate
       Con	Contract 15% Down payment regular terms
       ConLw	Contract Low Down payment and low interest
       ConLI	Contract Low Interest
       ConLD	Contract Low Down
       Oth	Other

In [ ]:
plot_count_box(data,'SaleType', 'SalePrice', orient = 'h')

In [ ]:
data['SaleType'].value_counts()

Too noisy. Let's drop it.

In [ ]:
cols_to_drop += ['SaleType']

SaleCondition: Condition of sale

       Normal	Normal Sale
       Abnorml	Abnormal Sale -  trade, foreclosure, short sale
       AdjLand	Adjoining Land Purchase
       Alloca	Allocation - two linked properties with separate deeds, typically condo with a garage unit	
       Family	Sale between family members
       Partial	Home was not completed when last assessed (associated with New Homes)

In [ ]:
plot_count_box(data,'SaleCondition', 'SalePrice')

High overlap.

In [ ]:
cols_to_drop += ['SaleCondition']

In [ ]:
cols_to_drop

In [ ]:
data.drop(columns=cols_to_drop, axis = 1, inplace = True)

In [ ]:
data.info()

# Inspecting numerical columns

In [ ]:
num_cols = data.select_dtypes(np.number)
num_cols.drop(columns='Id', axis = 1, inplace = True)
num_cols.head()

## Dealing with years

In [ ]:
year_cols = [ col for col in num_cols if (col.startswith('Year') or col.startswith('Yr'))]
year_cols

In [ ]:
data[year_cols].head()

In [ ]:
data[year_cols].isna().sum()

In [ ]:
data['YrSold'] - data['YearRemodAdd']

Let's change this columns to datetime

In [ ]:
data['Antiquity'] = data['YrSold'] - data['YearRemodAdd']
data.drop(columns = year_cols, axis = 1, inplace = True)

In [ ]:
num_cols = data.select_dtypes(np.number)
num_cols.drop(columns='Id', axis = 1, inplace = True)
num_cols.head()

In [ ]:
abs(num_cols.corr())

In [ ]:
c = num_cols.corr()
fig_dims = (20, 20)
mask = np.triu(np.ones_like(c, dtype=bool))
fig, ax = plt.subplots(figsize=fig_dims)
sns.heatmap(c, annot=True,ax = ax, mask=mask)

## Filtering features based on the correlation matrix

As we are going to use a **linear model** to predict the sale price, we can use the correlation matrix which quantifies the **linear realtionship** ( it can be other types of relationships not covered by a linear model ) to discard features. 

We will do this in two steps: 

First, we wil discard all those features for which the absolute value between the feature and the **SalePrice** is smaller than 0.5 ( usually I chose 0.75, however not too much features have such a high correlation, therefore we are in the risk of ending with a very reduced set of features). 

Second, we will dicard those fetature which are highly correlated among themselves as they are providing the same amount of information.

In [ ]:
d = pd.DataFrame(np.abs(c.loc[:,'SalePrice']).sort_values(ascending=False)).reset_index()
d.columns = ['Column','SalePrice']
d

Let's create a new list of columns to drop.

In [ ]:
cols_to_drop = []
d[d['SalePrice'] < 0.5]['Column'].tolist()

In [ ]:
cols_to_drop += d[d['SalePrice'] < 0.5]['Column'].tolist()
cols_to_drop

In [ ]:
data.drop(columns=cols_to_drop, axis = 1, inplace = True)

In [ ]:
num_cols = data.select_dtypes(np.number)
num_cols.drop(columns='Id', axis = 1, inplace = True)
num_cols.head()

Place column SalePrice to the end

In [ ]:
num_cols = num_cols[[ col for col in num_cols.columns if col != 'SalePrice'] + ['SalePrice']]
num_cols

In [ ]:
c = num_cols.corr()
fig_dims = (20, 20)
mask = np.triu(np.ones_like(c, dtype=bool))
fig, ax = plt.subplots(figsize=fig_dims)
sns.heatmap(c, annot=True,ax = ax, mask=mask)

Droping redundant columns: ie. columns highly correlated among themselves ( coor > 0.75). From those pairs, we will drop the ones which are **less correlated** with the dependent variable.

corr(`1stFlrSF`,`TotalBsmtSF`) > 0.75 and 
corr(`1stFlrSF`,`SalePrice`) = 0.63 <  corr(`TotalBsmtSF``,`SalePrice`)

we will drop `1stFlrSF`

corr(`TotRmsAbvGrd`,`GrLivArea`) > 0.75 and 
corr(`TotRmsAbvGrd`,`SalePrice`) = 0.54 <  corr(`GrLivArea`,`SalePrice`)

therefore, we will drop `TotRmsAbvGrd`

corr(`GarageArea`,`GarageCars`) > 0.75 and 
corr(`GarageArea`,`SalePrice`) = 0.64 <  corr(`GarageCars`,`SalePrice`)

therefore, we will drop `GarageArea`

In [ ]:
#cols_to_drop = ['1stFlrSF','TotRmsAbvGrd','GarageArea']
#data.drop(columns=cols_to_drop,axis=1,inplace = True)
data = data[[ col for col in data.columns if col != 'SalePrice'] + ['SalePrice']]
data.head()

In [ ]:
data.columns.tolist()

# Saving changes

In [ ]:
data.to_csv('/content/drive/MyDrive/OLD_CURR/UNIT7/DAY1/transformed_data.csv',index=False)

# Modelling

In [ ]:
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pickle

trans = PowerTransformer()
model = LinearRegression()

# define X and y (features and target)
X = data.drop(columns = ['Id','SalePrice'])
y = np.log(data['SalePrice']) # The model will try to predict the log(SalePrice) instead of the real price

# data splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# transforming data
trans.fit(X_train)

# Saving the transformer
with open('/content/drive/MyDrive/OLD_CURR/UNIT7/DAY1/transformer.pkl', 'wb')  as f:
    pickle.dump(trans,f)

X_train = trans.transform(X_train)
X_test  = trans.transform(X_test)

X_train = pd.DataFrame(X_train, columns = X.columns)
X_test  = pd.DataFrame(X_test,  columns = X.columns)

X_train.to_csv('/content/drive/MyDrive/OLD_CURR/UNIT7/DAY1/X_train.csv')
X_test.to_csv('/content/drive/MyDrive/OLD_CURR/UNIT7/DAY1/X_test.csv')

# model
model.fit(X_train, y_train)

with open('/content/drive/MyDrive/OLD_CURR/UNIT7/DAY1/model.pkl', 'wb')  as f:
    pickle.dump(model,f)

y_pred_train = model.predict(X_train)
y_pred_test  = model.predict(X_test)


In [ ]:
# Model validation
def model_performance(y_train, y_pred_train, y_test, y_pred_test):

    ME_train = np.mean(np.exp(y_train)-np.exp(y_pred_train))
    ME_test  = np.mean(np.exp(y_test)-np.exp(y_pred_test))

    MAE_train = mean_absolute_error(np.exp(y_train),np.exp(y_pred_train))
    MAE_test  = mean_absolute_error(np.exp(y_test),np.exp(y_pred_test))

    MSE_train = mean_squared_error(np.exp(y_train),np.exp(y_pred_train))
    MSE_test  = mean_squared_error(np.exp(y_test),np.exp(y_pred_test))

    RMSE_train = np.sqrt(MSE_train)
    RMSE_test  = np.sqrt(MSE_test)

    MAPE_train = np.mean((np.abs(np.exp(y_train)-np.exp(y_pred_train)) / np.exp(y_train))* 100.)
    MAPE_test  = np.mean((np.abs(np.exp(y_test)-np.exp(y_pred_test)) / np.exp(y_test))* 100.)

    R2_train = r2_score(np.exp(y_train),np.exp(y_pred_train))
    R2_test  = r2_score(np.exp(y_test),np.exp(y_pred_test))

    performance = pd.DataFrame({'Error_metric': ['Mean error','Mean absolute error','Mean squared error',
                                             'Root mean squared error','Mean absolute percentual error',
                                             'R2'],
                            'Train': [ME_train, MAE_train, MSE_train, RMSE_train, MAPE_train, R2_train],
                            'Test' : [ME_test, MAE_test , MSE_test, RMSE_test, MAPE_test, R2_test]})

    pd.options.display.float_format = '{:.2f}'.format

    df_train = pd.DataFrame({'x': np.exp(y_train), 'y': np.exp(y_pred_train)})
    df_test  = pd.DataFrame({'x': np.exp(y_test),  'y': np.exp(y_pred_test)})

    return performance, df_train, df_test

In [ ]:
performance,_,_ = model_performance(y_train, y_pred_train, y_test, y_pred_test)
performance

In [ ]:
def show_errors(y_real_train, y_pred_train, y_real_test, y_pred_test):

    plt.style.use('seaborn') 

    fig, ax = plt.subplots(2,2,figsize=(10,10))
    
    ax[0,0].scatter(x = np.exp(y_real_train), y = np.exp(y_pred_train), c = 'green')
    ax[0,0].plot([0,700000], [0,700000], linestyle = '--',c = 'black')
    ax[0,0].set_xlim(0,700000)
    ax[0,0].set_ylim(0,700000)

    ax[0,0].xaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{int(x/1000)}K'))
    ax[0,0].yaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{int(x/1000)}K'))
    ax[0,0].set_title('Train set')

    ax[0,1].hist(x = np.exp(y_real_train)-np.exp(y_pred_train), bins = 50,color = 'green')
    ax[0,1].set_xlim(-200000,200000)
    ax[0,1].xaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{int(x/1000)}K'))
    ax[0,1].set_title('Train set')

    ax[1,0].scatter(x = np.exp(y_real_test), y = np.exp(y_pred_test), c = 'blue')
    ax[1,0].plot([0,700000], [0,700000], linestyle = '--',c = 'black')
    ax[1,0].set_xlim(0,700000)
    ax[1,0].set_ylim(0,700000)
    ax[1,0].xaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{int(x/1000)}K'))
    ax[1,0].yaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{int(x/1000)}K'))
    ax[1,0].set_title('Test set')   

    ax[1,1].hist(x = np.exp(y_real_test)-np.exp(y_pred_test), bins = 50,color = 'blue')
    ax[1,1].set_xlim(-200000,200000)
    ax[1,1].xaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{int(x/1000)}K'))
    ax[1,1].set_title('Test set')

    fig.tight_layout()
    pass

In [ ]:
show_errors(y_train, y_pred_train, y_test, y_pred_test)

# Recaping all the transformations

In [ ]:
'''
data['Antiquity'] = data['YrSold'] - data['YearRemodAdd']
data[data['LotArea'] < 25000 ]
cols_to_keep = ['Id',
 'OverallQual',
 'TotalBsmtSF',
 'GrLivArea',
 'FullBath',
 'KitchenQual',
 'GarageCars',
 'Antiquity',
 'SalePrice',]
data['KitchenQual'] = data['KitchenQual'].replace({'Fa': 1, 'TA': 1.61, 'Gd': 1.61 'Ex': 3.14}, inplace = True)
data = data[[ col for col in data.columns if col != 'SalePrice'] + ['SalePrice']]
'''

## Create a preprocess function

In [ ]:
def preprocessing(df):
    '''
    Function to trasnform an input dataframe.
    It will add another column named "Antiquity"
    Drop all the Year related columns
    Drop any column not in
    cols_to_keep = ['Id',
   'OverallQual',
   'TotalBsmtSF',
   'GrLivArea',
   'FullBath',
   'KitchenQual',
   'GarageCars',
   'Antiquity',
   'SalePrice']
    Replace values of variable KitchenQual to
    {'Fa': 1, 'TA': 1.61, 'Gd': 1.61, 'Ex': 3.14}
    Drop any row with 'LotArea' < 25000   
    Input: 
    df = Pandas DataFrame object
    Output:
    Transformed DataFrame
    '''
    df2 = df.copy()

    # Computing Antiquity
    df2['Antiquity'] = df2['YrSold'] - df2['YearRemodAdd']
    year_cols = ['YearBuilt', 'YearRemodAdd', 'YrSold']
    df2.drop(columns = year_cols, axis = 1, inplace = True)

    # Filtering by LotArea
    df2 = df2[df2['LotArea'] < 25000 ]

    # Droping unwanted columns
    cols_to_keep = ['Id',
   'OverallQual',
   'TotalBsmtSF',
   'GrLivArea',
   'FullBath',
   'KitchenQual',
   'GarageCars',
   'Antiquity',
   'SalePrice']
    df2 = df2[cols_to_keep]

    # Replacing KitchenQual values by numerical ones.
    df2['KitchenQual'] = df2['KitchenQual'].replace({'Fa': 1, 'TA': 1.61, 'Gd': 1.61, 'Ex': 3.14})

    return df2

## Apply the preprocess function

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer, make_column_transformer

data2 = pd.read_csv("/content/drive/MyDrive/OLD_CURR/UNIT7/DAY1/housing_prices.csv") 

data2 = preprocessing(data2)

data2.head()


In [ ]:
data2 == data

# Inspecting the model

## Getting the coeficients

In [ ]:
model.coef_

In [ ]:
coeficients = pd.DataFrame({'Column': list(X_train.columns), 'Coef': list(model.coef_)}).sort_values(by ='Coef', ascending= False)
coeficients

## Getting the slope

In [ ]:
model.intercept_

In fact, we can't interpret this as starting price as this is the model predicts the logarithm. Therefore, this is the logarithm of the starting price. 

In [ ]:
round(np.exp(model.intercept_),2)

# Creating your own transformer

You can create a transformer of your choice using your own function. Then you will have access to the methods:

* fit
* transform
* fit_transform
* inv_transform

In [ ]:
data3 = pd.read_csv("/content/drive/MyDrive/OLD_CURR/UNIT7/DAY1/housing_prices.csv") 

In [ ]:
from sklearn.preprocessing import FunctionTransformer

procTransformer = FunctionTransformer(preprocessing)

X_new = procTransformer.transform(data3)
X_new

In [ ]:
type(X_new)